In [1]:
import mne
#from mne_bids import (BIDSPath, read_raw_bids, print_dir_tree, make_report,
#                      find_matching_paths, get_entity_vals)
import numpy as np
import matplotlib.pyplot as plt
# import scipy as sp
# from scipy import signal
# import seaborn as sns
import pandas as pd
from pathlib import Path
import os
from time import time
from autoreject import AutoReject, Ransac, get_rejection_threshold

%matplotlib qt

In [2]:
# Initialize an empty dictionary for subject grouping
subject_groups = {}

# Add subjects to the dictionary based on their group
for i in range(1, 25):
    subject_groups[f"sub-{i:03d}"] = "htr" #Himalayan

for i in range(25, 56):
    subject_groups[f"sub-{i:03d}"] = "ctr" #Control

for i in range(56, 60):
    subject_groups[f"sub-{i:03d}"] = "tm" #Transcendental???

for i in range(60, 79):
    subject_groups[f"sub-{i:03d}"] = "vip" #Vipassana

for i in range(79, 99):
    subject_groups[f"sub-{i:03d}"] = "sny" #Shoonya Yoga


# Creating event id for each task and practice
event_id = {'htr/med1':1, 'htr/med2':2, 'htr/think1':3, 'htr/think2':4,
          'ctr/med1':5, 'ctr/med2':6, 'ctr/think1':7, 'ctr/think2':8,
          'tm/med1': 9, 'tm/med2' :10, 'tm/think1':11, 'tm/think2':12,
          'vip/med1':13, 'vip/med2':14, 'vip/think1':15, 'vip/think2':16,
          'sny/med1':17, 'sny/med2':18, 'sny/think1':19, 'sny/think2':20}




def load_task_data(bids_root, subject_id, tasks):
    """
    Load specific task EEG data for a subject from a BIDS-like dataset.
    
    Parameters:
    - bids_root: str, path to the root of the dataset.
    - subject_id: str, subject identifier (e.g., '088').
    - tasks: list of str, the task names to load (e.g., ['med2', 'think2']).
    
    Returns:
    - raw_dict: dict, containing MNE Raw objects for each task.
    """
    # Define the subject's EEG folder
    subject_path = os.path.join(bids_root, f"sub-{subject_id}", "eeg")
    
    if not os.path.exists(subject_path):
        raise FileNotFoundError(f"Subject folder not found: {subject_path}")
    
    raw_dict = {}  # Dictionary to hold Raw objects for each task
    
    for task in tasks:
        # Locate the BDF file for the specific task
        bdf_file = os.path.join(subject_path, f"sub-{subject_id}_task-{task}_eeg.bdf")
        if not os.path.exists(bdf_file):
            print(f"WARNING: BDF file not found for task '{task}'. Skipping...")
            continue
        
        # Load the BDF file
        raw = mne.io.read_raw_bdf(bdf_file, preload=True)
        print(f"Loaded BDF file for task '{task}': {bdf_file}")
        
        # Optionally, load metadata from JSON/TSV files
        # Metadata file paths
        json_file = os.path.join(subject_path, f"sub-{subject_id}_task-{task}_eeg.json")
        tsv_file = os.path.join(subject_path, f"sub-{subject_id}_task-{task}_channels.tsv")
        
        metadata = {}
        if os.path.exists(json_file):
            with open(json_file, 'r') as f:
                metadata['json'] = f.read()
            print(f"Loaded metadata JSON for task '{task}'")
        
        if os.path.exists(tsv_file):
            metadata['tsv'] = tsv_file  # Store TSV file path for later use if needed
            print(f"Loaded metadata TSV for task '{task}'")
        
        # Add the loaded data and metadata to the dictionary
        raw_dict[task] = {
            'raw': raw,
            'metadata': metadata
        }
    
    if not raw_dict:
        raise ValueError(f"No data was loaded for subject {subject_id} with tasks {tasks}.")
    
    return raw_dict


In [3]:
# Print out intiation text with path to the dataset and events
print("\n##### Initiating the preprocessing pipeline for the Braboszcz et al. (2017) dataset. ######")

bids_root = "/home/sivert/Documents/Specialization_project/Datasets/ds003969/"
subject_id = "077"
tasks = ["med2", "think2"]
subject_group = subject_groups['sub-'+subject_id]

print('\n##### Loading data #####')
# Load data
subject_data = load_task_data(bids_root, subject_id, tasks)

# Access raw data for a specific task
raw_med = subject_data['med2']['raw']
raw_med.set_channel_types({'EXG1': 'misc',
                    'EXG2': 'misc',
                    'EXG3': 'misc',
                    'EXG4': 'misc',
                    'EXG5': 'misc',
                    'EXG6': 'misc',
                    'EXG7': 'misc',
                    'EXG8': 'misc',
                    'GSR1': 'misc',
                    'GSR2': 'misc',
                    'Erg1': 'misc',
                    'Erg2': 'misc',
                    'Resp': 'bio',
                    'Plet': 'bio',
                    'Temp': 'bio'
})
# Drop non-eeg channels
#raw_med.pick('eeg')

raw_med.plot(highpass=1, lowpass=None, block=True)


from mne.channels import find_ch_adjacency
from mne.viz import plot_ch_adjacency




##### Initiating the preprocessing pipeline for the Braboszcz et al. (2017) dataset. ######

##### Loading data #####


FileNotFoundError: Subject folder not found: /home/sivert/Documents/Specialization_project/Datasets/ds003969/sub-077\eeg

In [ ]:
from utils.file_io import load_raw_data, load_bad_channels
from utils.preprocessing_tools import prepare_raw_data
from utils.config import DATASETS, set_plot_style, EEG_SETTINGS
import utils.config as config

set_plot_style()

DATASET = DATASETS['braboszcz2017']
path_derivatives = DATASET.path_derivatives
subject = '069'
task = 'think2'

bad_chs = load_bad_channels(
        save_dir=path_derivatives,
        dataset=DATASET.f_name,
        subject=subject,
        task=task
    )

raw = load_raw_data(dataset=DATASET, subject=subject, task=task)

raw = prepare_raw_data(raw, dataset=DATASET, eeg_settings=config.EEG_SETTINGS)

raw.info['bads'] = bad_chs

raw.plot(highpass=1, lowpass=None, block=True)


Found existing bad channels for sub-069 - task-think2
Extracting EDF parameters from C:\Users\si_ve\Documents\Master_AttentionalDirectionResearch\data\braboszcz2017\raw\sub-069\eeg\sub-069_task-think2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 618495  =      0.000 ...   603.999 secs...
Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)


C:\Users\si_ve\Documents\Master_AttentionalDirectionResearch\utils\preprocessing_tools.py:52: RuntimeWarning: The unit for channel(s) EXG1, EXG2, EXG3, EXG4, EXG5, EXG6, EXG7, EXG8, Erg1, Erg2, GSR1, GSR2 has changed from V to NA.
  raw.set_channel_types({'EXG1': 'misc',


Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 1.00 Hz: -6.02 dB



c:\Users\si_ve\anaconda3\envs\mne\Lib\site-packages\mne_qt_browser\_pg_figure.py:4372: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z = zscore(data, axis=1)


Channels marked as bad:
['Fp1', 'FC3', 'T7', 'PO3', 'O1', 'POz', 'F4', 'FT8', 'FC4', 'FC2', 'P10', 'PO8', 'PO4', 'O2']


In [5]:
from utils.file_io import load_raw_data, load_bad_channels
from utils.preprocessing_tools import prepare_raw_data
from utils.config import DATASETS
import utils.config as config

DATASET = DATASETS['jin2019']
path_derivatives = DATASET.path_derivatives
subject = 9
session = 1

bad_chs = load_bad_channels(
        save_dir=path_derivatives,
        dataset=DATASET.f_name,
        subject=subject,
        session=session
    )

raw = load_raw_data(dataset=DATASET, subject=subject, session=session)

raw = prepare_raw_data(raw, dataset=DATASET, eeg_settings=config.EEG_SETTINGS)

raw.info['bads'] = bad_chs

raw.plot(highpass=1, lowpass=None, block=True)


Found existing bad channels for sub-9 - ses-1
Extracting EDF parameters from C:\Users\si_ve\Documents\Master_AttentionalDirectionResearch\data\jin2019\raw\sub9_1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3100671  =      0.000 ...  6055.998 secs...


C:\Users\si_ve\Documents\Master_AttentionalDirectionResearch\utils\preprocessing_tools.py:92: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw.set_channel_types({


Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
831 events found on stim channel Status
Event IDs: [   10    11    12    13    14    20    21    30    40   254 65790]
Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
831 events found on stim channel Status
Event IDs: [   10    11    12    13    14    20    21    30    40   254 65790]
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 1.00 Hz: -6.02 dB

Channels marked as bad:
['P2']
